<a href="https://colab.research.google.com/github/Pushkar0655g/Generative-AI/blob/main/MULTI_40enhanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install dependencies
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q yt-dlp
!pip install -q transformers
!apt-get install -y ffmpeg -qq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
# Import necessary libraries
from google.colab import files
import whisper
import subprocess
import os
from transformers import MarianMTModel, MarianTokenizer, AutoTokenizer, AutoModelForSeq2SeqLM

# Define default video paths
DOWNLOADED_VIDEO_PATH = "/content/video.mp4"
UPLOADED_VIDEO_PATH = "/content/uploaded_video.mp4"

# Function to download video from YouTube
def download_youtube_video(url, output_path):
    cookies_path = "/content/cookies.txt"
    use_cookies = False

    print("If the video requires sign-in (e.g., age-restricted), upload a cookies file (optional).")
    print("Guide: https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp")
    uploaded = files.upload()
    if "cookies.txt" in uploaded:
        with open(cookies_path, "wb") as f:
            f.write(uploaded["cookies.txt"])
        use_cookies = True
        print("Cookies uploaded successfully.")

    try:
        command = [
            "yt-dlp",
            "--user-agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "-o", output_path,
            "--merge-output-format", "mp4",
        ]
        if use_cookies:
            command.extend(["--cookies", cookies_path])
        command.append(url)

        result = subprocess.run(command, capture_output=True, text=True, check=True)
        print(f"Downloaded video to {output_path}")
        if not os.path.exists(output_path):
            print(f"Error: File {output_path} was not created. Check yt-dlp output:")
            print(f"Output: {result.stdout}")
            print(f"Errors: {result.stderr}")
            return False
        if use_cookies and os.path.exists(cookies_path):
            os.remove(cookies_path)
        return True

    except subprocess.CalledProcessError as e:
        print(f"Failed to download video from {url}:")
        print(f"Error output: {e.stderr}")
        if "Sign in to confirm you’re not a bot" in e.stderr or "age-restricted" in e.stderr:
            print("\n⚠️ This video requires authentication (e.g., sign-in or age verification).")
            if not use_cookies:
                print("You didn’t upload cookies. Options:")
                print("1. Upload a cookies.txt file (restart and try again).")
                print("2. Try a different public video.")
                print("3. Download locally with yt-dlp and upload to Colab.")
            else:
                print("Cookies provided but still failed. Ensure they’re valid and from a signed-in browser.")
        else:
            print("Unexpected error. Check the URL or try again.")
        if use_cookies and os.path.exists(cookies_path):
            os.remove(cookies_path)
        return False

# Load Whisper model
model = whisper.load_model("base")

# Function to create an SRT file
def create_srt(segments, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for i, segment in enumerate(segments, 1):
            start = f"{segment['start']:.3f}".replace(".", ",")
            end = f"{segment['end']:.3f}".replace(".", ",")
            text = segment["text"].strip()
            f.write(f"{i}\n00:00:{start} --> 00:00:{end}\n{text}\n\n")
    print(f"Subtitles saved to {filename}")

# Function to process video and generate subtitles
def process_video(video_path, language):
    if not os.path.exists(video_path):
        print(f"Error: Video file {video_path} does not exist.")
        return None

    try:
        print("Transcribing video to English...")
        result = model.transcribe(video_path, language="en")

        if language == "english":
            segments = result["segments"]
        else:
            segments = []
            if language == "telugu":
                model_name = "facebook/nllb-200-distilled-600M"
                tokenizer = AutoTokenizer.from_pretrained(model_name)
                translation_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
                tgt_lang = "tel_Telu"
                print(f"Translating to Telugu using NLLB-200 Distilled...")
                for segment in result["segments"]:
                    inputs = tokenizer(segment["text"], return_tensors="pt", padding=True)
                    translated_tokens = translation_model.generate(**inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang))
                    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
                    segments.append({"text": translated_text, "start": segment["start"], "end": segment["end"]})
            else:
                model_map = {
                    "hindi": "Helsinki-NLP/opus-mt-en-hi",
                    "spanish": "Helsinki-NLP/opus-mt-en-es",
                    "french": "Helsinki-NLP/opus-mt-en-fr",
                    "german": "Helsinki-NLP/opus-mt-en-de",
                    "portuguese": "Helsinki-NLP/opus-mt-en-pt",
                    "russian": "Helsinki-NLP/opus-mt-en-ru",
                    "chinese": "Helsinki-NLP/opus-mt-en-zh",
                    "arabic": "Helsinki-NLP/opus-mt-en-ar",
                    "japanese": "Helsinki-NLP/opus-mt-en-jap"
                }
                model_name = model_map.get(language)
                if not model_name:
                    print(f"Language '{language}' not supported.")
                    return None

                tokenizer = MarianTokenizer.from_pretrained(model_name)
                translation_model = MarianMTModel.from_pretrained(model_name)
                print(f"Translating to {language.capitalize()}...")
                for segment in result["segments"]:
                    inputs = tokenizer(segment["text"], return_tensors="pt", padding=True)
                    translated = translation_model.generate(**inputs)
                    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
                    segments.append({"text": translated_text, "start": segment["start"], "end": segment["end"]})

        srt_path = f"/content/subtitles_{language}.srt"
        create_srt(segments, srt_path)
        return srt_path

    except Exception as e:
        print(f"Error processing video: {str(e)}")
        return None

# Main function
def main():
    print("Choose the source of the video:")
    print("1. YouTube")
    print("2. Upload from Local Computer")
    choice = input("Enter your choice (1 or 2): ").strip()

    if choice == "1":
        youtube_link = input("Enter the YouTube video URL: ").strip()
        if not youtube_link:
            print("Error: YouTube URL cannot be empty.")
            return
        success = download_youtube_video(youtube_link, DOWNLOADED_VIDEO_PATH)
        if not success:
            print("Failed to download YouTube video. Review the error above for next steps.")
            return
        final_video_path = DOWNLOADED_VIDEO_PATH
    elif choice == "2":
        print("Uploading video from local computer...")
        uploaded = files.upload()
        if not uploaded:
            print("No file uploaded. Exiting.")
            return
        filename = next(iter(uploaded))
        with open(UPLOADED_VIDEO_PATH, "wb") as f:
            f.write(uploaded[filename])
        print(f"File '{filename}' uploaded successfully.")
        final_video_path = UPLOADED_VIDEO_PATH
    else:
        print("Invalid choice.")
        return

    print("\nChoose the language for subtitles:")
    print("1. English")
    print("2. Hindi")
    print("3. Spanish")
    print("4. French")
    print("5. German")
    print("6. Telugu")
    print("7. Portuguese")
    print("8. Russian")
    print("9. Chinese")
    print("10. Arabic")
    print("11. Japanese")
    language_choice = input("Enter your choice (1-11): ").strip()

    language_map = {
        "1": "english",
        "2": "hindi",
        "3": "spanish",
        "4": "french",
        "5": "german",
        "6": "telugu",
        "7": "portuguese",
        "8": "russian",
        "9": "chinese",
        "10": "arabic",
        "11": "japanese"
    }

    if language_choice not in language_map:
        print("Invalid choice.")
        return

    language = language_map[language_choice]
    print(f"\nStarting video processing for {language} subtitles...")
    srt_path = process_video(final_video_path, language)

    if srt_path:
        print(f"\nSample of {language} subtitles:")
        with open(srt_path, "r", encoding="utf-8") as f:
            print(f.read(500))
        files.download(srt_path)
        # Cleanup after processing
        if final_video_path in [DOWNLOADED_VIDEO_PATH, UPLOADED_VIDEO_PATH] and os.path.exists(final_video_path):
            os.remove(final_video_path)
            print(f"Cleaned up: Removed {final_video_path}")
    else:
        print("Video failed to process. Please try again.")

# Run the main function
if __name__ == "__main__":
    main()

Choose the source of the video:
1. YouTube
2. Upload from Local Computer
Enter your choice (1 or 2): 1
Enter the YouTube video URL: https://youtu.be/5BBTghKXjDE?si=ntY8lJ6ZPwog5MOX
If the video requires sign-in (e.g., age-restricted), upload a cookies file (optional).
Guide: https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp


Saving cookies.txt to cookies.txt
Cookies uploaded successfully.
Downloaded video to /content/video.mp4

Choose the language for subtitles:
1. English
2. Hindi
3. Spanish
4. French
5. German
6. Telugu
7. Portuguese
8. Russian
9. Chinese
10. Arabic
11. Japanese
Enter your choice (1-11): 8

Starting video processing for russian subtitles...
Transcribing video to English...


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Translating to Russian...
Subtitles saved to /content/subtitles_russian.srt

Sample of russian subtitles:
1
00:00:0,000 --> 00:00:4,400
То, что у нас в гонке между Бугатти Сиро и Супер Спортом, и Конекс, возьмите да.

2
00:00:4,400 --> 00:00:7,200
Ну, я собираюсь выяснить это, гоня их на стоящую четверть мили.

3
00:00:7,200 --> 00:00:12,160
Я Аутер, а ты смотришь "Машину Вау" и там могут быть другие интересные машины.

4
00:00:12,160 --> 00:00:15,440
Да, это безумие.

5
00:00:19,360 --> 00:00:27,440
Bugatti Siro и Super Sport имеют 8,0 л турбо W16 1600HP, 1600 Нм крутящего момента

6
00:00:27,440 -


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cleaned up: Removed /content/video.mp4
